In [1]:
import sys
import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
from pathlib import Path
import shutil


from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))

In [16]:
ANNEX_regexp= r'^(ANNEX)\ *([0-9.]|ONE|TWO|THREE|FOUR|FIVE|SIX|SEVEN|EIGHT|NINE|TEN|ELEVEN|TWELVE|THIRTEEN|FOURTEEN|FIFTEEN|SIXTEEN|SEVENTEEN|EIGHTEEN|NINETEEN|TWENTY|I|II|III|IV|V|VI|VII|VIII|IX|X|X(I|II|III|IV|V|VI|VII|VIII|IX|X))* *$'

import sys
from base64 import b64encode
from base64 import b64decode


sys.path.append(  "/notebook/nas-trainings/arne/DGFISMA/definition_extraction/notebooks/" )

import requests
from cleaning import clean_html


#read in (train data)
data=pd.read_csv(  "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/Milieu_GOLDEN_STANDARD/Milieu_iteratie_1.csv"  , sep=',' , header=None ) 

https=data[0].tolist()
labels=data[1].tolist()

labels=[ 1 if label.strip()=='Accepted' else 0 for label in labels   ]

assert( len( https ) == len( labels )  )

from bs4 import BeautifulSoup

all_articles=[]

for i,http in enumerate(https):

    page_response=requests.get( http )

    articles=clean_html( page_response.content.decode( 'utf-8' ) )

    #delete the annexes:
    articles_no_annexes=[]
    for article in articles:
        if bool(re.match( ANNEX_regexp,  article.strip("\n").split( "\n" )[0]  , re.IGNORECASE )):
            #continue
            break  #not adding annexes, or anything that comes after annex to the training set
        else:
            articles_no_annexes.append( article )

    if articles_no_annexes:
        delimiter=100*"■"
        article_no_annexes=delimiter.join(  articles_no_annexes  )
        all_articles.append( article_no_annexes )
        
    print(i)

assert( len( all_articles  ) == len(labels) )


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


In [24]:
random.seed( 10 )

labels_text=[]
for label, text in zip( labels, all_articles  ):
    labels_text.append(  ( label, text  ) )

random.shuffle( labels_text )

with open( os.path.join(  "/notebook/nas-trainings/arne/DGFISMA/DATA/doc_classifier/processed_eurlex_15_06", "test_data_milieu_iteration1.tsv"  ) , 'w'  ) as f:
    for item in labels_text:
        if item[0]==1:
            label_name='accepted'
        else:
            label_name='declined'

        encoded_doc=b64encode(  item[1].encode() )
        f.write( f"{encoded_doc.decode()  }\t{ label_name }\t{item[0]}\n" )

# END

In [12]:
import random
from sklearn import metrics


vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                         stop_words= 'english' )

feature_selection=SelectFromModel(LinearSVC(penalty= "l1" , dual=False,
                                                          tol=1e-3, class_weight='balanced'  )) 

classifier=LinearSVC(penalty="l1", loss="squared_hinge" , dual=False )
#calibrated the classifier (for predict_proba): 
calibrated_classifier = CalibratedClassifierCV(classifier , cv=5 ) 


clf=Pipeline([
( 'vectorizer', vectorizer)  ,
#( 'chisquare', ch2  )  ,
('feature_selection',  feature_selection  )   ,
('classification', calibrated_classifier  )
])


random.seed( 10 )


labels_text=[]
for label, text in zip( labels, all_articles  ):
    labels_text.append(  ( label, text  ) )
    

random.shuffle( labels_text )


print( len(labels_text) )

train_set=labels_text[:80]
test_set=labels_text[ 80: ]

train_labels=[ item[0] for item in train_set   ]
train_data=[ item[1] for item in train_set   ]

test_labels=[ item[0] for item in test_set   ]
test_data=[ item[1] for item in test_set   ]

clf.fit(  train_data , train_labels )

pred=clf.predict( test_data  )

print( metrics.classification_report( test_labels , pred ) )



117
              precision    recall  f1-score   support

           0       0.82      0.96      0.88        24
           1       0.89      0.62      0.73        13

   micro avg       0.84      0.84      0.84        37
   macro avg       0.86      0.79      0.81        37
weighted avg       0.85      0.84      0.83        37

